# Import libraries and data

In [1]:
# Import libraries
import os
import openai
import pandas as pd
import numpy as np
import openpyxl

In [2]:
# Get OpenAI API secret key (change this file with your own OpenAI key)
open_ai_key = open('../data/local/openai_key.txt', 'r').read()
openai.api_key = open_ai_key
#openai.Model.list()

In [3]:
# Prompt for GPT
prompt_raw = "I would like to summarise in maximum 300 characters and in English what the following sentences in Portuguese talk about. \
Do not use aspects mentioned only in one sentence as part of the summary: TWEETS"

# 2016 negative

## January

In [4]:
# Get monthly data
df_2016_01_neg = pd.read_csv('../data/local/df_neg_2016-01.csv').reset_index()
df_2016_01_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1231 entries, 0 to 1230
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1231 non-null   int64  
 1   LocationCode            1231 non-null   object 
 2   LocationName            1231 non-null   object 
 3   StatesCode              1231 non-null   object 
 4   ...4                    1231 non-null   int64  
 5   id                      1231 non-null   float64
 6   text                    1231 non-null   object 
 7   label                   1231 non-null   object 
 8   in_reply_to_status_id   141 non-null    float64
 9   in_reply_to_user_id     160 non-null    float64
 10  quoted_user_id          36 non-null     float64
 11  quoted_status_id        36 non-null     float64
 12  retweeted_user_id       239 non-null    float64
 13  retweeted_status_id     239 non-null    float64
 14  created_at              1231 non-null   

### Representative sample based on week and location

In [5]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2016_01_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2016_01_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2016-w01,Acre,4,0.003249
1,2016-w01,Alagoas,3,0.002437
2,2016-w01,Amapa,1,0.000812
3,2016-w01,Amazonas,4,0.003249
4,2016-w01,Bahia,9,0.007311
...,...,...,...,...
96,2016-w05,Rio De Janeiro,1,0.000812
97,2016-w05,Rio Grande Do Sul,4,0.003249
98,2016-w05,Roraima,1,0.000812
99,2016-w05,Santa Catarina,1,0.000812


In [6]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2016-w01,Acre,4,0.003249,7
1,2016-w01,Alagoas,3,0.002437,5
2,2016-w01,Amapa,1,0.000812,1
3,2016-w01,Amazonas,4,0.003249,7
4,2016-w01,Bahia,9,0.007311,17
...,...,...,...,...,...
96,2016-w05,Rio De Janeiro,1,0.000812,1
97,2016-w05,Rio Grande Do Sul,4,0.003249,7
98,2016-w05,Roraima,1,0.000812,1
99,2016-w05,Santa Catarina,1,0.000812,1


In [7]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2016_01_neg_sample_size = pd.merge(df_2016_01_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [8]:
# Step 6: Perform the stratified sampling
#df_2016_01_neg_sample = df_2016_01_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2016_01_neg_sample = df_2016_01_neg
# Step 7: Check the result
print(f"Sampled {len(df_2016_01_neg_sample)} rows.")


Sampled 1231 rows.


In [9]:
text_gpt_2016_01_neg = '\n'.join(df_2016_01_neg_sample['text'].astype(str))

### Prompts and loops

In [10]:
prompt_1 = prompt_raw.replace('TWEETS', text_gpt_2016_01_neg)

In [11]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_1}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [12]:
response.choices[0].message.content

'The sentences discuss various concerns and experiences related to vaccines, highlighting issues such as pain from vaccination, fear of needles, and reactions to vaccines. Many express frustration over the lack of vaccines in public health facilities, the high costs of certain vaccines, and doubts about their efficacy. Additionally, there are mentions of public health policies, the impact of political decisions on vaccination availability, and concerns about potential side effects, including a controversial association of certain vaccines with health issues like microcephaly.'

In [13]:
summary = []
dataset = []

In [14]:
dataset.append("2016_01_negitive")

In [15]:
summary.append(response.choices[0].message.content)

In [16]:
df_2016_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2016_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  1 non-null      object
 1   summary  1 non-null      object
dtypes: object(2)
memory usage: 144.0+ bytes


In [17]:
df_2016_neg.tail(14)

,dataset,summary
0,2016_01_negitive,The sentences discuss various concerns and exp...


In [18]:
response

<OpenAIObject chat.completion id=chatcmpl-AUcx0fe0s6FKbhroBHXFuhpbsmaPD at 0x174d7c76d10> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The sentences discuss various concerns and experiences related to vaccines, highlighting issues such as pain from vaccination, fear of needles, and reactions to vaccines. Many express frustration over the lack of vaccines in public health facilities, the high costs of certain vaccines, and doubts about their efficacy. Additionally, there are mentions of public health policies, the impact of political decisions on vaccination availability, and concerns about potential side effects, including a controversial association of certain vaccines with health issues like microcephaly.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731862930,
  "id": "chatcmpl-AUcx0fe0s6FKbhroBHXFuhpbsmaPD",
  "model": "gpt-4o-mini-2024-07-18",
 

## February

In [19]:
# Get monthly data
df_2016_02_neg = pd.read_csv('../data/local/df_neg_2016-02.csv').reset_index()
df_2016_02_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1390 entries, 0 to 1389
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1390 non-null   int64  
 1   LocationCode            1390 non-null   object 
 2   LocationName            1390 non-null   object 
 3   StatesCode              1390 non-null   object 
 4   ...4                    1390 non-null   int64  
 5   id                      1390 non-null   float64
 6   text                    1390 non-null   object 
 7   label                   1390 non-null   object 
 8   in_reply_to_status_id   172 non-null    float64
 9   in_reply_to_user_id     186 non-null    float64
 10  quoted_user_id          53 non-null     float64
 11  quoted_status_id        53 non-null     float64
 12  retweeted_user_id       410 non-null    float64
 13  retweeted_status_id     410 non-null    float64
 14  created_at              1390 non-null   

### Representative sample based on week and location

In [20]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2016_02_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2016_02_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2016-w05,Alagoas,2,0.001439
1,2016-w05,Amazonas,3,0.002158
2,2016-w05,Bahia,25,0.017986
3,2016-w05,Ceara,6,0.004317
4,2016-w05,Distrito Federal,8,0.005755
...,...,...,...,...
99,2016-w09,Roraima,2,0.001439
100,2016-w09,Santa Catarina,6,0.004317
101,2016-w09,Sao Paulo,18,0.012950
102,2016-w09,Sergipe,1,0.000719


In [21]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2016-w05,Alagoas,2,0.001439,3
1,2016-w05,Amazonas,3,0.002158,5
2,2016-w05,Bahia,25,0.017986,44
3,2016-w05,Ceara,6,0.004317,10
4,2016-w05,Distrito Federal,8,0.005755,14
...,...,...,...,...,...
99,2016-w09,Roraima,2,0.001439,3
100,2016-w09,Santa Catarina,6,0.004317,10
101,2016-w09,Sao Paulo,18,0.012950,31
102,2016-w09,Sergipe,1,0.000719,1


In [22]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2016_02_neg_sample_size = pd.merge(df_2016_02_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [23]:
# Step 6: Perform the stratified sampling
#df_2016_02_neg_sample = df_2016_02_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2016_02_neg_sample = df_2016_02_neg
# Step 7: Check the result
print(f"Sampled {len(df_2016_02_neg_sample)} rows.")


Sampled 1390 rows.


In [24]:
text_gpt_2016_02_neg = '\n'.join(df_2016_02_neg_sample['text'].astype(str))

### Prompts and loops

In [25]:
prompt_2 = prompt_raw.replace('TWEETS', text_gpt_2016_02_neg)

In [26]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_2}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [27]:
response.choices[0].message.content

"The text discusses various concerns surrounding vaccines in Brazil, particularly focusing on the HPV vaccine, microcephaly linked to vaccination, and the slow development of vaccines against diseases like Zika. There's public outcry about the lack of vaccines, allegations of vaccine-related health issues, and accusations against the government for mismanagement and lack of funding for vaccine research. The sentiment reflects a mix of skepticism, fear, and frustration regarding vaccination policies and their implications for public health."

In [28]:
#summary = []
#dataset = []

In [29]:
dataset.append("2016_02_negitive")

In [30]:
summary.append(response.choices[0].message.content)

In [31]:
df_2016_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2016_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  2 non-null      object
 1   summary  2 non-null      object
dtypes: object(2)
memory usage: 160.0+ bytes


In [32]:
df_2016_neg.tail(14)

,dataset,summary
0,2016_01_negitive,The sentences discuss various concerns and exp...
1,2016_02_negitive,The text discusses various concerns surroundin...


In [33]:
response

<OpenAIObject chat.completion id=chatcmpl-AUcx3SbaEx389FoIftu176AMi3syM at 0x174d7cbcef0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various concerns surrounding vaccines in Brazil, particularly focusing on the HPV vaccine, microcephaly linked to vaccination, and the slow development of vaccines against diseases like Zika. There's public outcry about the lack of vaccines, allegations of vaccine-related health issues, and accusations against the government for mismanagement and lack of funding for vaccine research. The sentiment reflects a mix of skepticism, fear, and frustration regarding vaccination policies and their implications for public health.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731862933,
  "id": "chatcmpl-AUcx3SbaEx389FoIftu176AMi3syM",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "syst

## March

In [34]:
# Get monthly data
df_2016_03_neg = pd.read_csv('../data/local/df_neg_2016-03.csv').reset_index()
df_2016_03_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 849 entries, 0 to 848
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   849 non-null    int64  
 1   LocationCode            849 non-null    object 
 2   LocationName            849 non-null    object 
 3   StatesCode              849 non-null    object 
 4   ...4                    849 non-null    int64  
 5   id                      849 non-null    float64
 6   text                    849 non-null    object 
 7   label                   849 non-null    object 
 8   in_reply_to_status_id   95 non-null     float64
 9   in_reply_to_user_id     106 non-null    float64
 10  quoted_user_id          21 non-null     float64
 11  quoted_status_id        21 non-null     float64
 12  retweeted_user_id       107 non-null    float64
 13  retweeted_status_id     107 non-null    float64
 14  created_at              849 non-null    ob

### Representative sample based on week and location

In [35]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2016_03_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2016_03_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2016-w09,Amazonas,2,0.002356
1,2016-w09,Bahia,4,0.004711
2,2016-w09,Ceara,2,0.002356
3,2016-w09,Espirito Santo,2,0.002356
4,2016-w09,Goias,15,0.017668
...,...,...,...,...
89,2016-w13,Rio Grande Do Sul,21,0.024735
90,2016-w13,Rondonia,1,0.001178
91,2016-w13,Santa Catarina,16,0.018846
92,2016-w13,Sao Paulo,62,0.073027


In [36]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2016-w09,Amazonas,2,0.002356,5
1,2016-w09,Bahia,4,0.004711,11
2,2016-w09,Ceara,2,0.002356,5
3,2016-w09,Espirito Santo,2,0.002356,5
4,2016-w09,Goias,15,0.017668,43
...,...,...,...,...,...
89,2016-w13,Rio Grande Do Sul,21,0.024735,60
90,2016-w13,Rondonia,1,0.001178,2
91,2016-w13,Santa Catarina,16,0.018846,46
92,2016-w13,Sao Paulo,62,0.073027,178


In [37]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2016_03_neg_sample_size = pd.merge(df_2016_03_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [39]:
# Step 6: Perform the stratified sampling
#df_2016_03_neg_sample = df_2016_03_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2016_03_neg_sample = df_2016_03_neg
# Step 7: Check the result
print(f"Sampled {len(df_2016_03_neg_sample)} rows.")


Sampled 849 rows.


In [40]:
text_gpt_2016_03_neg = '\n'.join(df_2016_03_neg_sample['text'].astype(str))

### Prompts and loops

In [41]:
prompt_3 = prompt_raw.replace('TWEETS', text_gpt_2016_03_neg)

In [42]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_3}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [43]:
response.choices[0].message.content

'The text discusses various opinions and experiences related to vaccines, highlighting fears, discomfort, and societal issues surrounding vaccination campaigns. Many express dislike for vaccines, recount painful experiences, and mention shortages or lack of availability. There are also concerns about vaccine safety and effectiveness, particularly regarding specific diseases like H1N1 and Zika.'

In [44]:
#summary = []
#dataset = []

In [45]:
dataset.append("2016_03_negitive")

In [46]:
summary.append(response.choices[0].message.content)

In [47]:
df_2016_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2016_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  3 non-null      object
 1   summary  3 non-null      object
dtypes: object(2)
memory usage: 176.0+ bytes


In [48]:
df_2016_neg.tail(14)

,dataset,summary
0,2016_01_negitive,The sentences discuss various concerns and exp...
1,2016_02_negitive,The text discusses various concerns surroundin...
2,2016_03_negitive,The text discusses various opinions and experi...


In [49]:
response

<OpenAIObject chat.completion id=chatcmpl-AUcxs04FSTXBkEup5d4YJKcJb2rtn at 0x174d7f8e220> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various opinions and experiences related to vaccines, highlighting fears, discomfort, and societal issues surrounding vaccination campaigns. Many express dislike for vaccines, recount painful experiences, and mention shortages or lack of availability. There are also concerns about vaccine safety and effectiveness, particularly regarding specific diseases like H1N1 and Zika.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731862984,
  "id": "chatcmpl-AUcxs04FSTXBkEup5d4YJKcJb2rtn",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_9b78b61c52",
  "usage": {
    "completion_tokens": 64,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,

## April

In [50]:
# Get monthly data
df_2016_04_neg = pd.read_csv('../data/local/df_neg_2016-04.csv').reset_index()
df_2016_04_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5205 entries, 0 to 5204
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   5205 non-null   int64  
 1   LocationCode            5205 non-null   object 
 2   LocationName            5205 non-null   object 
 3   StatesCode              5205 non-null   object 
 4   ...4                    5205 non-null   int64  
 5   id                      5205 non-null   float64
 6   text                    5205 non-null   object 
 7   label                   5205 non-null   object 
 8   in_reply_to_status_id   356 non-null    float64
 9   in_reply_to_user_id     415 non-null    float64
 10  quoted_user_id          88 non-null     float64
 11  quoted_status_id        88 non-null     float64
 12  retweeted_user_id       932 non-null    float64
 13  retweeted_status_id     932 non-null    float64
 14  created_at              5205 non-null   

### Representative sample based on week and location

In [51]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2016_04_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2016_04_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2016-w14,Acre,1,0.000192
1,2016-w14,Alagoas,9,0.001729
2,2016-w14,Amapa,2,0.000384
3,2016-w14,Amazonas,7,0.001345
4,2016-w14,Bahia,44,0.008453
...,...,...,...,...
115,2016-w18,Rondonia,5,0.000961
116,2016-w18,Roraima,3,0.000576
117,2016-w18,Santa Catarina,20,0.003842
118,2016-w18,Sao Paulo,119,0.022863


In [52]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2016-w14,Acre,1,0.000192,0
1,2016-w14,Alagoas,9,0.001729,4
2,2016-w14,Amapa,2,0.000384,0
3,2016-w14,Amazonas,7,0.001345,3
4,2016-w14,Bahia,44,0.008453,20
...,...,...,...,...,...
115,2016-w18,Rondonia,5,0.000961,2
116,2016-w18,Roraima,3,0.000576,1
117,2016-w18,Santa Catarina,20,0.003842,9
118,2016-w18,Sao Paulo,119,0.022863,56


In [53]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2016_04_neg_sample_size = pd.merge(df_2016_04_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [54]:
# Step 6: Perform the stratified sampling
df_2016_04_neg_sample = df_2016_04_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
).reset_index(drop=True)

# Step 7: Check the result
print(f"Sampled {len(df_2016_04_neg_sample)} rows.")


Sampled 2385 rows.


In [55]:
text_gpt_2016_04_neg = '\n'.join(df_2016_04_neg_sample['text'].astype(str))

### Prompts and loops

In [56]:
prompt_4 = prompt_raw.replace('TWEETS', text_gpt_2016_04_neg)

In [57]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_4}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [58]:
response.choices[0].message.content

"The text discusses various issues related to the H1N1 vaccination in Brazil, including shortages of vaccines, rising costs, and public frustration. It highlights incidents where vaccinations were mishandled, such as patients receiving insulin instead of the vaccine. There's a strong sentiment against mandatory vaccinations, fears of side effects, and criticism of government priorities, especially regarding the vaccination of prisoners over the general public."

In [59]:
#summary = []
#dataset = []

In [60]:
dataset.append("2016_04_negitive")

In [61]:
summary.append(response.choices[0].message.content)

In [62]:
df_2016_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2016_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  4 non-null      object
 1   summary  4 non-null      object
dtypes: object(2)
memory usage: 192.0+ bytes


In [63]:
df_2016_neg.tail(14)

,dataset,summary
0,2016_01_negitive,The sentences discuss various concerns and exp...
1,2016_02_negitive,The text discusses various concerns surroundin...
2,2016_03_negitive,The text discusses various opinions and experi...
3,2016_04_negitive,The text discusses various issues related to t...


In [64]:
response

<OpenAIObject chat.completion id=chatcmpl-AUcxxIlWJo4lXip9DhtXUTmO9VsO3 at 0x174db5dbf40> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various issues related to the H1N1 vaccination in Brazil, including shortages of vaccines, rising costs, and public frustration. It highlights incidents where vaccinations were mishandled, such as patients receiving insulin instead of the vaccine. There's a strong sentiment against mandatory vaccinations, fears of side effects, and criticism of government priorities, especially regarding the vaccination of prisoners over the general public.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731862989,
  "id": "chatcmpl-AUcxxIlWJo4lXip9DhtXUTmO9VsO3",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 78,
    "

## May

In [65]:
# Get monthly data
df_2016_05_neg = pd.read_csv('../data/local/df_neg_2016-05.csv').reset_index()
df_2016_05_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4588 entries, 0 to 4587
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   4588 non-null   int64  
 1   LocationCode            4588 non-null   object 
 2   LocationName            4588 non-null   object 
 3   StatesCode              4588 non-null   object 
 4   ...4                    4588 non-null   int64  
 5   id                      4588 non-null   float64
 6   text                    4588 non-null   object 
 7   label                   4588 non-null   object 
 8   in_reply_to_status_id   323 non-null    float64
 9   in_reply_to_user_id     375 non-null    float64
 10  quoted_user_id          133 non-null    float64
 11  quoted_status_id        133 non-null    float64
 12  retweeted_user_id       583 non-null    float64
 13  retweeted_status_id     583 non-null    float64
 14  created_at              4588 non-null   

### Representative sample based on week and location

In [66]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2016_05_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2016_05_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2016-w18,Acre,1,0.000218
1,2016-w18,Alagoas,6,0.001308
2,2016-w18,Amapa,8,0.001744
3,2016-w18,Amazonas,17,0.003705
4,2016-w18,Bahia,65,0.014167
...,...,...,...,...
111,2016-w22,Roraima,1,0.000218
112,2016-w22,Santa Catarina,9,0.001962
113,2016-w22,Sao Paulo,61,0.013296
114,2016-w22,Sergipe,1,0.000218


In [67]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2016-w18,Acre,1,0.000218,0
1,2016-w18,Alagoas,6,0.001308,3
2,2016-w18,Amapa,8,0.001744,4
3,2016-w18,Amazonas,17,0.003705,9
4,2016-w18,Bahia,65,0.014167,34
...,...,...,...,...,...
111,2016-w22,Roraima,1,0.000218,0
112,2016-w22,Santa Catarina,9,0.001962,4
113,2016-w22,Sao Paulo,61,0.013296,32
114,2016-w22,Sergipe,1,0.000218,0


In [68]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2016_05_neg_sample_size = pd.merge(df_2016_05_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [69]:
# Step 6: Perform the stratified sampling
df_2016_05_neg_sample = df_2016_05_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
).reset_index(drop=True)

# Step 7: Check the result
print(f"Sampled {len(df_2016_05_neg_sample)} rows.")


Sampled 2398 rows.


In [70]:
text_gpt_2016_05_neg = '\n'.join(df_2016_05_neg_sample['text'].astype(str))

### Prompts and loops

In [71]:
prompt_5 = prompt_raw.replace('TWEETS', text_gpt_2016_05_neg)

In [72]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_5}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [73]:
response.choices[0].message.content

'The text discusses various concerns and experiences related to vaccinations, particularly for the H1N1 flu and other vaccines. Many individuals express fear, anxiety, and physical discomfort after receiving vaccines, often highlighting pain in their arms and side effects. There are complaints about the lack of vaccine availability in health centers, dissatisfaction with government actions regarding vaccination campaigns, and criticism of prioritizing certain groups over others. Overall, the sentiments reflect frustration, fear of needles, and the societal issues surrounding vaccination accessibility.'

In [74]:
#summary = []
#dataset = []

In [75]:
dataset.append("2016_05_negitive")

In [76]:
summary.append(response.choices[0].message.content)

In [77]:
df_2016_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2016_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  5 non-null      object
 1   summary  5 non-null      object
dtypes: object(2)
memory usage: 208.0+ bytes


In [78]:
df_2016_neg.tail(14)

,dataset,summary
0,2016_01_negitive,The sentences discuss various concerns and exp...
1,2016_02_negitive,The text discusses various concerns surroundin...
2,2016_03_negitive,The text discusses various opinions and experi...
3,2016_04_negitive,The text discusses various issues related to t...
4,2016_05_negitive,The text discusses various concerns and experi...


In [79]:
response

<OpenAIObject chat.completion id=chatcmpl-AUcy6F7OV3XJRttI1h2yJ1xItyoDU at 0x174dc999770> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various concerns and experiences related to vaccinations, particularly for the H1N1 flu and other vaccines. Many individuals express fear, anxiety, and physical discomfort after receiving vaccines, often highlighting pain in their arms and side effects. There are complaints about the lack of vaccine availability in health centers, dissatisfaction with government actions regarding vaccination campaigns, and criticism of prioritizing certain groups over others. Overall, the sentiments reflect frustration, fear of needles, and the societal issues surrounding vaccination accessibility.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731862998,
  "id": "chatcmpl-AUcy6F7OV3XJRttI1h2yJ1xItyoDU",
  "model": "

## June

In [80]:
# Get monthly data
df_2016_06_neg = pd.read_csv('../data/local/df_neg_2016-06.csv').reset_index()
df_2016_06_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1244 entries, 0 to 1243
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1244 non-null   int64  
 1   LocationCode            1244 non-null   object 
 2   LocationName            1244 non-null   object 
 3   StatesCode              1244 non-null   object 
 4   ...4                    1244 non-null   int64  
 5   id                      1244 non-null   float64
 6   text                    1244 non-null   object 
 7   label                   1244 non-null   object 
 8   in_reply_to_status_id   116 non-null    float64
 9   in_reply_to_user_id     119 non-null    float64
 10  quoted_user_id          30 non-null     float64
 11  quoted_status_id        30 non-null     float64
 12  retweeted_user_id       116 non-null    float64
 13  retweeted_status_id     116 non-null    float64
 14  created_at              1244 non-null   

### Representative sample based on week and location

In [81]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2016_06_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2016_06_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2016-w22,Amapa,2,0.001608
1,2016-w22,Amazonas,1,0.000804
2,2016-w22,Bahia,6,0.004823
3,2016-w22,Ceara,1,0.000804
4,2016-w22,Distrito Federal,6,0.004823
...,...,...,...,...
94,2016-w26,Rondonia,1,0.000804
95,2016-w26,Roraima,1,0.000804
96,2016-w26,Santa Catarina,7,0.005627
97,2016-w26,Sao Paulo,28,0.022508


In [82]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2016-w22,Amapa,2,0.001608,3
1,2016-w22,Amazonas,1,0.000804,1
2,2016-w22,Bahia,6,0.004823,11
3,2016-w22,Ceara,1,0.000804,1
4,2016-w22,Distrito Federal,6,0.004823,11
...,...,...,...,...,...
94,2016-w26,Rondonia,1,0.000804,1
95,2016-w26,Roraima,1,0.000804,1
96,2016-w26,Santa Catarina,7,0.005627,13
97,2016-w26,Sao Paulo,28,0.022508,55


In [83]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2016_06_neg_sample_size = pd.merge(df_2016_06_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [85]:
# Step 6: Perform the stratified sampling
#df_2016_06_neg_sample = df_2016_06_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2016_06_neg_sample = df_2016_06_neg
# Step 7: Check the result
print(f"Sampled {len(df_2016_06_neg_sample)} rows.")


Sampled 1244 rows.


In [86]:
text_gpt_2016_06_neg = '\n'.join(df_2016_06_neg_sample['text'].astype(str))

### Prompts and loops

In [87]:
prompt_6 = prompt_raw.replace('TWEETS', text_gpt_2016_06_neg)

In [88]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_6}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [89]:
response.choices[0].message.content

'The text discusses various opinions and experiences related to vaccinations, particularly in Brazil. Many express fear, pain, and frustration associated with getting vaccinated, often highlighting side effects and feeling unwell afterward. There are also concerns about the availability of vaccines, particularly in Acre, which has the lowest vaccination coverage for influenza in the country. Some comments reflect a distrust of vaccines and the healthcare system, while others emphasize the necessity of vaccination for public health.'

In [90]:
#summary = []
#dataset = []

In [91]:
dataset.append("2016_06_negitive")

In [92]:
summary.append(response.choices[0].message.content)

In [93]:
df_2016_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2016_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  6 non-null      object
 1   summary  6 non-null      object
dtypes: object(2)
memory usage: 224.0+ bytes


In [94]:
df_2016_neg.tail(14)

,dataset,summary
0,2016_01_negitive,The sentences discuss various concerns and exp...
1,2016_02_negitive,The text discusses various concerns surroundin...
2,2016_03_negitive,The text discusses various opinions and experi...
3,2016_04_negitive,The text discusses various issues related to t...
4,2016_05_negitive,The text discusses various concerns and experi...
5,2016_06_negitive,The text discusses various opinions and experi...


In [95]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd0kNuGhXLYsLnps5hCg2FkcDUkz at 0x174dceb0bd0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various opinions and experiences related to vaccinations, particularly in Brazil. Many express fear, pain, and frustration associated with getting vaccinated, often highlighting side effects and feeling unwell afterward. There are also concerns about the availability of vaccines, particularly in Acre, which has the lowest vaccination coverage for influenza in the country. Some comments reflect a distrust of vaccines and the healthcare system, while others emphasize the necessity of vaccination for public health.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863162,
  "id": "chatcmpl-AUd0kNuGhXLYsLnps5hCg2FkcDUkz",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_finge

## July

In [96]:
# Get monthly data
df_2016_07_neg = pd.read_csv('../data/local/df_neg_2016-07.csv').reset_index()
df_2016_07_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1074 entries, 0 to 1073
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1074 non-null   int64  
 1   LocationCode            1074 non-null   object 
 2   LocationName            1074 non-null   object 
 3   StatesCode              1074 non-null   object 
 4   ...4                    1074 non-null   int64  
 5   id                      1074 non-null   float64
 6   text                    1074 non-null   object 
 7   label                   1074 non-null   object 
 8   in_reply_to_status_id   95 non-null     float64
 9   in_reply_to_user_id     99 non-null     float64
 10  quoted_user_id          34 non-null     float64
 11  quoted_status_id        34 non-null     float64
 12  retweeted_user_id       140 non-null    float64
 13  retweeted_status_id     140 non-null    float64
 14  created_at              1074 non-null   

### Representative sample based on week and location

In [97]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2016_07_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2016_07_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2016-w27,Alagoas,1,0.000931
1,2016-w27,Amapa,1,0.000931
2,2016-w27,Amazonas,5,0.004655
3,2016-w27,Bahia,18,0.016760
4,2016-w27,Ceara,1,0.000931
...,...,...,...,...
89,2016-w31,Roraima,2,0.001862
90,2016-w31,Santa Catarina,1,0.000931
91,2016-w31,Sao Paulo,23,0.021415
92,2016-w31,Sergipe,1,0.000931


In [98]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2016-w27,Alagoas,1,0.000931,2
1,2016-w27,Amapa,1,0.000931,2
2,2016-w27,Amazonas,5,0.004655,11
3,2016-w27,Bahia,18,0.016760,41
4,2016-w27,Ceara,1,0.000931,2
...,...,...,...,...,...
89,2016-w31,Roraima,2,0.001862,4
90,2016-w31,Santa Catarina,1,0.000931,2
91,2016-w31,Sao Paulo,23,0.021415,52
92,2016-w31,Sergipe,1,0.000931,2


In [99]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2016_07_neg_sample_size = pd.merge(df_2016_07_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [100]:
# Step 6: Perform the stratified sampling
#df_2016_07_neg_sample = df_2016_07_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2016_07_neg_sample = df_2016_07_neg
# Step 7: Check the result
print(f"Sampled {len(df_2016_07_neg_sample)} rows.")


Sampled 1074 rows.


In [101]:
text_gpt_2016_07_neg = '\n'.join(df_2016_07_neg_sample['text'].astype(str))

### Prompts and loops

In [102]:
prompt_7 = prompt_raw.replace('TWEETS', text_gpt_2016_07_neg)

In [103]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_7}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [104]:
response.choices[0].message.content

'The text discusses various opinions and reactions to vaccinations, highlighting fears, pain, and skepticism about their efficacy, particularly regarding the dengue vaccine in Brazil. Many express discomfort and negative experiences related to vaccinations, alongside criticisms of government actions regarding vaccine availability and costs. There are also humorous takes on the situation, emphasizing the societal attitudes toward vaccines and the emotional responses they provoke.'

In [105]:
#summary = []
#dataset = []

In [106]:
dataset.append("2016_07_negitive")

In [107]:
summary.append(response.choices[0].message.content)

In [108]:
df_2016_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2016_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  7 non-null      object
 1   summary  7 non-null      object
dtypes: object(2)
memory usage: 240.0+ bytes


In [109]:
df_2016_neg.tail(14)

,dataset,summary
0,2016_01_negitive,The sentences discuss various concerns and exp...
1,2016_02_negitive,The text discusses various concerns surroundin...
2,2016_03_negitive,The text discusses various opinions and experi...
3,2016_04_negitive,The text discusses various issues related to t...
4,2016_05_negitive,The text discusses various concerns and experi...
5,2016_06_negitive,The text discusses various opinions and experi...
6,2016_07_negitive,The text discusses various opinions and reacti...


In [110]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd0nsU63f1fMJ3A6H0hrvakl7BQ1 at 0x174dcf68770> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various opinions and reactions to vaccinations, highlighting fears, pain, and skepticism about their efficacy, particularly regarding the dengue vaccine in Brazil. Many express discomfort and negative experiences related to vaccinations, alongside criticisms of government actions regarding vaccine availability and costs. There are also humorous takes on the situation, emphasizing the societal attitudes toward vaccines and the emotional responses they provoke.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863165,
  "id": "chatcmpl-AUd0nsU63f1fMJ3A6H0hrvakl7BQ1",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion

## August

In [111]:
# Get monthly data
df_2016_08_neg = pd.read_csv('../data/local/df_neg_2016-08.csv').reset_index()
df_2016_08_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947 entries, 0 to 946
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   947 non-null    int64  
 1   LocationCode            947 non-null    object 
 2   LocationName            947 non-null    object 
 3   StatesCode              947 non-null    object 
 4   ...4                    947 non-null    int64  
 5   id                      947 non-null    float64
 6   text                    947 non-null    object 
 7   label                   947 non-null    object 
 8   in_reply_to_status_id   70 non-null     float64
 9   in_reply_to_user_id     73 non-null     float64
 10  quoted_user_id          22 non-null     float64
 11  quoted_status_id        22 non-null     float64
 12  retweeted_user_id       104 non-null    float64
 13  retweeted_status_id     104 non-null    float64
 14  created_at              947 non-null    ob

### Representative sample based on week and location

In [112]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2016_08_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2016_08_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2016-w31,Acre,1,0.001056
1,2016-w31,Alagoas,1,0.001056
2,2016-w31,Amazonas,4,0.004224
3,2016-w31,Bahia,10,0.010560
4,2016-w31,Ceara,3,0.003168
...,...,...,...,...
87,2016-w35,Rio De Janeiro,1,0.001056
88,2016-w35,Rio Grande Do Sul,18,0.019007
89,2016-w35,Rondonia,2,0.002112
90,2016-w35,Santa Catarina,4,0.004224


In [113]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2016-w31,Acre,1,0.001056,2
1,2016-w31,Alagoas,1,0.001056,2
2,2016-w31,Amazonas,4,0.004224,10
3,2016-w31,Bahia,10,0.010560,25
4,2016-w31,Ceara,3,0.003168,7
...,...,...,...,...,...
87,2016-w35,Rio De Janeiro,1,0.001056,2
88,2016-w35,Rio Grande Do Sul,18,0.019007,46
89,2016-w35,Rondonia,2,0.002112,5
90,2016-w35,Santa Catarina,4,0.004224,10


In [114]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2016_08_neg_sample_size = pd.merge(df_2016_08_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [116]:
# Step 6: Perform the stratified sampling
#df_2016_08_neg_sample = df_2016_08_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2016_08_neg_sample = df_2016_08_neg
# Step 7: Check the result
print(f"Sampled {len(df_2016_08_neg_sample)} rows.")


Sampled 947 rows.


In [117]:
text_gpt_2016_08_neg = '\n'.join(df_2016_08_neg_sample['text'].astype(str))

### Prompts and loops

In [118]:
prompt_8 = prompt_raw.replace('TWEETS', text_gpt_2016_08_neg)

In [119]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_8}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [120]:
response.choices[0].message.content

'The text discusses various opinions and experiences regarding vaccines, particularly the dengue vaccine, which is criticized for its high prices (ranging from R$ 750 to R$ 915) and painful side effects. Many express a strong dislike for vaccinations in general, recounting their personal discomfort and fears related to getting vaccinated, while also highlighting the financial burden and accessibility issues associated with vaccines.'

In [121]:
#summary = []
#dataset = []

In [122]:
dataset.append("2016_08_negitive")

In [123]:
summary.append(response.choices[0].message.content)

In [124]:
df_2016_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2016_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  8 non-null      object
 1   summary  8 non-null      object
dtypes: object(2)
memory usage: 256.0+ bytes


In [125]:
df_2016_neg.tail(14)

,dataset,summary
0,2016_01_negitive,The sentences discuss various concerns and exp...
1,2016_02_negitive,The text discusses various concerns surroundin...
2,2016_03_negitive,The text discusses various opinions and experi...
3,2016_04_negitive,The text discusses various issues related to t...
4,2016_05_negitive,The text discusses various concerns and experi...
5,2016_06_negitive,The text discusses various opinions and experi...
6,2016_07_negitive,The text discusses various opinions and reacti...
7,2016_08_negitive,The text discusses various opinions and experi...


In [126]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd28KWvaWoFEPu9PdI4NFf46lA9g at 0x174dce06310> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various opinions and experiences regarding vaccines, particularly the dengue vaccine, which is criticized for its high prices (ranging from R$ 750 to R$ 915) and painful side effects. Many express a strong dislike for vaccinations in general, recounting their personal discomfort and fears related to getting vaccinated, while also highlighting the financial burden and accessibility issues associated with vaccines.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863248,
  "id": "chatcmpl-AUd28KWvaWoFEPu9PdI4NFf46lA9g",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 76,
    "completion_tokens_details": 

## September

In [127]:
# Get monthly data
df_2016_09_neg = pd.read_csv('../data/local/df_neg_2016-09.csv').reset_index()
df_2016_09_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1478 entries, 0 to 1477
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1478 non-null   int64  
 1   LocationCode            1478 non-null   object 
 2   LocationName            1478 non-null   object 
 3   StatesCode              1478 non-null   object 
 4   ...4                    1478 non-null   int64  
 5   id                      1478 non-null   float64
 6   text                    1478 non-null   object 
 7   label                   1478 non-null   object 
 8   in_reply_to_status_id   123 non-null    float64
 9   in_reply_to_user_id     128 non-null    float64
 10  quoted_user_id          27 non-null     float64
 11  quoted_status_id        27 non-null     float64
 12  retweeted_user_id       148 non-null    float64
 13  retweeted_status_id     148 non-null    float64
 14  created_at              1478 non-null   

### Representative sample based on week and location

In [128]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2016_09_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2016_09_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2016-w35,Bahia,1,0.000677
1,2016-w35,Distrito Federal,1,0.000677
2,2016-w35,Maranhao,1,0.000677
3,2016-w35,Mato Grosso,1,0.000677
4,2016-w35,Mato Grosso Do Sul,11,0.007442
...,...,...,...,...
96,2016-w40,Paraiba,3,0.002030
97,2016-w40,Parana,1,0.000677
98,2016-w40,Rio Grande Do Sul,9,0.006089
99,2016-w40,Santa Catarina,1,0.000677


In [129]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2016-w35,Bahia,1,0.000677,1
1,2016-w35,Distrito Federal,1,0.000677,1
2,2016-w35,Maranhao,1,0.000677,1
3,2016-w35,Mato Grosso,1,0.000677,1
4,2016-w35,Mato Grosso Do Sul,11,0.007442,18
...,...,...,...,...,...
96,2016-w40,Paraiba,3,0.002030,4
97,2016-w40,Parana,1,0.000677,1
98,2016-w40,Rio Grande Do Sul,9,0.006089,14
99,2016-w40,Santa Catarina,1,0.000677,1


In [130]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2016_09_neg_sample_size = pd.merge(df_2016_09_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [131]:
# Step 6: Perform the stratified sampling
#df_2016_09_neg_sample = df_2016_09_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2016_09_neg_sample = df_2016_09_neg
# Step 7: Check the result
print(f"Sampled {len(df_2016_09_neg_sample)} rows.")


Sampled 1478 rows.


In [132]:
text_gpt_2016_09_neg = '\n'.join(df_2016_09_neg_sample['text'].astype(str))

### Prompts and loops

In [133]:
prompt_9 = prompt_raw.replace('TWEETS', text_gpt_2016_09_neg)

In [134]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_9}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [135]:
response.choices[0].message.content

'The sentences express a strong aversion to vaccinations, highlighting fears, pains, and negative experiences associated with them. Many mention physical discomfort following vaccinations, such as sore arms and reactions like fever. There are also concerns about vaccine shortages and safety, particularly regarding pets. The overall sentiment reflects anxiety and disdain for the vaccination process.'

In [136]:
#summary = []
#dataset = []

In [137]:
dataset.append("2016_09_negitive")

In [138]:
summary.append(response.choices[0].message.content)

In [139]:
df_2016_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2016_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  9 non-null      object
 1   summary  9 non-null      object
dtypes: object(2)
memory usage: 272.0+ bytes


In [140]:
df_2016_neg.tail(14)

,dataset,summary
0,2016_01_negitive,The sentences discuss various concerns and exp...
1,2016_02_negitive,The text discusses various concerns surroundin...
2,2016_03_negitive,The text discusses various opinions and experi...
3,2016_04_negitive,The text discusses various issues related to t...
4,2016_05_negitive,The text discusses various concerns and experi...
5,2016_06_negitive,The text discusses various opinions and experi...
6,2016_07_negitive,The text discusses various opinions and reacti...
7,2016_08_negitive,The text discusses various opinions and experi...
8,2016_09_negitive,The sentences express a strong aversion to vac...


In [141]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd2BYWQZ9VqgmSZrNDGAAgvnldxN at 0x174dcd8f4a0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The sentences express a strong aversion to vaccinations, highlighting fears, pains, and negative experiences associated with them. Many mention physical discomfort following vaccinations, such as sore arms and reactions like fever. There are also concerns about vaccine shortages and safety, particularly regarding pets. The overall sentiment reflects anxiety and disdain for the vaccination process.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863251,
  "id": "chatcmpl-AUd2BYWQZ9VqgmSZrNDGAAgvnldxN",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 64,
    "completion_tokens_details": {
      "accepted_prediction_tokens

## October

In [142]:
# Get monthly data
df_2016_10_neg = pd.read_csv('../data/local/df_neg_2016-10.csv').reset_index()
df_2016_10_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826 entries, 0 to 825
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   826 non-null    int64  
 1   LocationCode            826 non-null    object 
 2   LocationName            826 non-null    object 
 3   StatesCode              826 non-null    object 
 4   ...4                    826 non-null    int64  
 5   id                      826 non-null    float64
 6   text                    826 non-null    object 
 7   label                   826 non-null    object 
 8   in_reply_to_status_id   97 non-null     float64
 9   in_reply_to_user_id     98 non-null     float64
 10  quoted_user_id          12 non-null     float64
 11  quoted_status_id        12 non-null     float64
 12  retweeted_user_id       73 non-null     float64
 13  retweeted_status_id     73 non-null     float64
 14  created_at              826 non-null    ob

### Representative sample based on week and location

In [143]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2016_10_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2016_10_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2016-w40,Amazonas,7,0.008475
1,2016-w40,Bahia,7,0.008475
2,2016-w40,Ceara,3,0.003632
3,2016-w40,Distrito Federal,5,0.006053
4,2016-w40,Espirito Santo,6,0.007264
...,...,...,...,...
85,2016-w44,Pernambuco,1,0.001211
86,2016-w44,Rio De Janeiro,1,0.001211
87,2016-w44,Rio Grande Do Sul,8,0.009685
88,2016-w44,Santa Catarina,3,0.003632


In [144]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2016-w40,Amazonas,7,0.008475,20
1,2016-w40,Bahia,7,0.008475,20
2,2016-w40,Ceara,3,0.003632,8
3,2016-w40,Distrito Federal,5,0.006053,14
4,2016-w40,Espirito Santo,6,0.007264,17
...,...,...,...,...,...
85,2016-w44,Pernambuco,1,0.001211,2
86,2016-w44,Rio De Janeiro,1,0.001211,2
87,2016-w44,Rio Grande Do Sul,8,0.009685,23
88,2016-w44,Santa Catarina,3,0.003632,8


In [145]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2016_10_neg_sample_size = pd.merge(df_2016_10_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [146]:
# Step 6: Perform the stratified sampling
#df_2016_10_neg_sample = df_2016_10_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2016_10_neg_sample = df_2016_10_neg
# Step 7: Check the result
print(f"Sampled {len(df_2016_10_neg_sample)} rows.")


Sampled 826 rows.


In [147]:
text_gpt_2016_10_neg = '\n'.join(df_2016_10_neg_sample['text'].astype(str))

### Prompts and loops

In [148]:
prompt_10 = prompt_raw.replace('TWEETS', text_gpt_2016_10_neg)

In [149]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_10}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [150]:
response.choices[0].message.content

'The provided sentences discuss a variety of experiences and sentiments related to vaccinations. Common themes include fear and anxiety about receiving vaccines, physical pain and side effects afterward, and frustrations with the healthcare system regarding vaccine availability. Some also express skepticism or negative beliefs about vaccines, while others share humorous or relatable anecdotes about the challenges of vaccination, especially for children and pets.'

In [151]:
#summary = []
#dataset = []

In [152]:
dataset.append("2016_10_negitive")

In [153]:
summary.append(response.choices[0].message.content)

In [154]:
df_2016_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2016_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  10 non-null     object
 1   summary  10 non-null     object
dtypes: object(2)
memory usage: 288.0+ bytes


In [155]:
df_2016_neg.tail(14)

,dataset,summary
0,2016_01_negitive,The sentences discuss various concerns and exp...
1,2016_02_negitive,The text discusses various concerns surroundin...
2,2016_03_negitive,The text discusses various opinions and experi...
3,2016_04_negitive,The text discusses various issues related to t...
4,2016_05_negitive,The text discusses various concerns and experi...
5,2016_06_negitive,The text discusses various opinions and experi...
6,2016_07_negitive,The text discusses various opinions and reacti...
7,2016_08_negitive,The text discusses various opinions and experi...
8,2016_09_negitive,The sentences express a strong aversion to vac...
9,2016_10_negitive,The provided sentences discuss a variety of ex...


In [156]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd2E0gwnnbh73I8vvmDCxdtLW4rM at 0x174de13ee00> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The provided sentences discuss a variety of experiences and sentiments related to vaccinations. Common themes include fear and anxiety about receiving vaccines, physical pain and side effects afterward, and frustrations with the healthcare system regarding vaccine availability. Some also express skepticism or negative beliefs about vaccines, while others share humorous or relatable anecdotes about the challenges of vaccination, especially for children and pets.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863254,
  "id": "chatcmpl-AUd2E0gwnnbh73I8vvmDCxdtLW4rM",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 70,
   

## November

In [157]:
# Get monthly data
df_2016_11_neg = pd.read_csv('../data/local/df_neg_2016-11.csv').reset_index()
df_2016_11_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 716 entries, 0 to 715
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   716 non-null    int64  
 1   LocationCode            716 non-null    object 
 2   LocationName            716 non-null    object 
 3   StatesCode              716 non-null    object 
 4   ...4                    716 non-null    int64  
 5   id                      716 non-null    float64
 6   text                    716 non-null    object 
 7   label                   716 non-null    object 
 8   in_reply_to_status_id   79 non-null     float64
 9   in_reply_to_user_id     81 non-null     float64
 10  quoted_user_id          15 non-null     float64
 11  quoted_status_id        15 non-null     float64
 12  retweeted_user_id       100 non-null    float64
 13  retweeted_status_id     100 non-null    float64
 14  created_at              716 non-null    ob

### Representative sample based on week and location

In [158]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2016_11_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2016_11_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2016-w44,Alagoas,1,0.001397
1,2016-w44,Amazonas,2,0.002793
2,2016-w44,Bahia,3,0.004190
3,2016-w44,Ceara,3,0.004190
4,2016-w44,Distrito Federal,6,0.008380
...,...,...,...,...
72,2016-w48,Parana,6,0.008380
73,2016-w48,Rio Grande Do Sul,13,0.018156
74,2016-w48,Santa Catarina,8,0.011173
75,2016-w48,Sao Paulo,37,0.051676


In [159]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2016-w44,Alagoas,1,0.001397,3
1,2016-w44,Amazonas,2,0.002793,6
2,2016-w44,Bahia,3,0.004190,10
3,2016-w44,Ceara,3,0.004190,10
4,2016-w44,Distrito Federal,6,0.008380,20
...,...,...,...,...,...
72,2016-w48,Parana,6,0.008380,20
73,2016-w48,Rio Grande Do Sul,13,0.018156,44
74,2016-w48,Santa Catarina,8,0.011173,27
75,2016-w48,Sao Paulo,37,0.051676,126


In [160]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2016_11_neg_sample_size = pd.merge(df_2016_11_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [161]:
# Step 6: Perform the stratified sampling
#df_2016_11_neg_sample = df_2016_11_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2016_11_neg_sample = df_2016_11_neg
# Step 7: Check the result
print(f"Sampled {len(df_2016_11_neg_sample)} rows.")


Sampled 716 rows.


In [162]:
text_gpt_2016_11_neg = '\n'.join(df_2016_11_neg_sample['text'].astype(str))

### Prompts and loops

In [163]:
prompt_11 = prompt_raw.replace('TWEETS', text_gpt_2016_11_neg)

In [164]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_11}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [165]:
response.choices[0].message.content

"The text discusses various aspects of vaccination, highlighting concerns about expired vaccines being administered to infants and the general discomfort and fear associated with vaccinations. It also mentions the importance of animal vaccinations and the public's frustrations regarding vaccine availability and side effects. There are references to public health issues, misinformation surrounding vaccines, and a mix of humorous and serious reactions from individuals about their vaccination experiences."

In [166]:
#summary = []
#dataset = []

In [167]:
dataset.append("2016_11_negitive")

In [168]:
summary.append(response.choices[0].message.content)

In [169]:
df_2016_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2016_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  11 non-null     object
 1   summary  11 non-null     object
dtypes: object(2)
memory usage: 304.0+ bytes


In [170]:
df_2016_neg.tail(14)

,dataset,summary
0,2016_01_negitive,The sentences discuss various concerns and exp...
1,2016_02_negitive,The text discusses various concerns surroundin...
2,2016_03_negitive,The text discusses various opinions and experi...
3,2016_04_negitive,The text discusses various issues related to t...
4,2016_05_negitive,The text discusses various concerns and experi...
5,2016_06_negitive,The text discusses various opinions and experi...
6,2016_07_negitive,The text discusses various opinions and reacti...
7,2016_08_negitive,The text discusses various opinions and experi...
8,2016_09_negitive,The sentences express a strong aversion to vac...
9,2016_10_negitive,The provided sentences discuss a variety of ex...


In [171]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd2HCwkyT2J1fAZysfkx5IVQQHjR at 0x174de300d60> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various aspects of vaccination, highlighting concerns about expired vaccines being administered to infants and the general discomfort and fear associated with vaccinations. It also mentions the importance of animal vaccinations and the public's frustrations regarding vaccine availability and side effects. There are references to public health issues, misinformation surrounding vaccines, and a mix of humorous and serious reactions from individuals about their vaccination experiences.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863257,
  "id": "chatcmpl-AUd2HCwkyT2J1fAZysfkx5IVQQHjR",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "u

## December

In [172]:
# Get monthly data
df_2016_12_neg = pd.read_csv('../data/local/df_neg_2016-12.csv').reset_index()
df_2016_12_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 633 entries, 0 to 632
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   633 non-null    int64  
 1   LocationCode            633 non-null    object 
 2   LocationName            633 non-null    object 
 3   StatesCode              633 non-null    object 
 4   ...4                    633 non-null    int64  
 5   id                      633 non-null    float64
 6   text                    633 non-null    object 
 7   label                   633 non-null    object 
 8   in_reply_to_status_id   80 non-null     float64
 9   in_reply_to_user_id     84 non-null     float64
 10  quoted_user_id          14 non-null     float64
 11  quoted_status_id        14 non-null     float64
 12  retweeted_user_id       96 non-null     float64
 13  retweeted_status_id     96 non-null     float64
 14  created_at              633 non-null    ob

### Representative sample based on week and location

In [173]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2016_12_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2016_12_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2016-w48,Bahia,1,0.001580
1,2016-w48,Mato Grosso Do Sul,1,0.001580
2,2016-w48,Minas Gerais,10,0.015798
3,2016-w48,Paraiba,1,0.001580
4,2016-w48,Parana,2,0.003160
...,...,...,...,...
77,2016-w53,Goias,1,0.001580
78,2016-w53,Maranhao,1,0.001580
79,2016-w53,Minas Gerais,9,0.014218
80,2016-w53,Rio Grande Do Sul,1,0.001580


In [174]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2016-w48,Bahia,1,0.001580,3
1,2016-w48,Mato Grosso Do Sul,1,0.001580,3
2,2016-w48,Minas Gerais,10,0.015798,38
3,2016-w48,Paraiba,1,0.001580,3
4,2016-w48,Parana,2,0.003160,7
...,...,...,...,...,...
77,2016-w53,Goias,1,0.001580,3
78,2016-w53,Maranhao,1,0.001580,3
79,2016-w53,Minas Gerais,9,0.014218,34
80,2016-w53,Rio Grande Do Sul,1,0.001580,3


In [175]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2016_12_neg_sample_size = pd.merge(df_2016_12_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [176]:
# Step 6: Perform the stratified sampling
#df_2016_12_neg_sample = df_2016_12_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2016_12_neg_sample = df_2016_12_neg
# Step 7: Check the result
print(f"Sampled {len(df_2016_12_neg_sample)} rows.")


Sampled 633 rows.


In [177]:
text_gpt_2016_12_neg = '\n'.join(df_2016_12_neg_sample['text'].astype(str))

### Prompts and loops

In [178]:
prompt_12 = prompt_raw.replace('TWEETS', text_gpt_2016_12_neg)

In [179]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_12}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [180]:
response.choices[0].message.content

'The text discusses various personal experiences and opinions regarding vaccinations, primarily highlighting the common fears, pains, and discomfort associated with getting vaccinated. It also touches on skepticism about vaccines, humorous comparisons, and mentions issues like allergic reactions in pets and problems with vaccine supply. Overall, it reflects a mix of anxiety, frustration, and occasional humor about the vaccination process.'

In [181]:
#summary = []
#dataset = []

In [182]:
dataset.append("2016_12_negitive")

In [183]:
summary.append(response.choices[0].message.content)

In [184]:
df_2016_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2016_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  12 non-null     object
 1   summary  12 non-null     object
dtypes: object(2)
memory usage: 320.0+ bytes


In [185]:
df_2016_neg.tail(14)

,dataset,summary
0,2016_01_negitive,The sentences discuss various concerns and exp...
1,2016_02_negitive,The text discusses various concerns surroundin...
2,2016_03_negitive,The text discusses various opinions and experi...
3,2016_04_negitive,The text discusses various issues related to t...
4,2016_05_negitive,The text discusses various concerns and experi...
5,2016_06_negitive,The text discusses various opinions and experi...
6,2016_07_negitive,The text discusses various opinions and reacti...
7,2016_08_negitive,The text discusses various opinions and experi...
8,2016_09_negitive,The sentences express a strong aversion to vac...
9,2016_10_negitive,The provided sentences discuss a variety of ex...


In [186]:
response

<OpenAIObject chat.completion id=chatcmpl-AUd2KfWZb4GIVRa0HuIHYqn6KnKJI at 0x174de302860> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various personal experiences and opinions regarding vaccinations, primarily highlighting the common fears, pains, and discomfort associated with getting vaccinated. It also touches on skepticism about vaccines, humorous comparisons, and mentions issues like allergic reactions in pets and problems with vaccine supply. Overall, it reflects a mix of anxiety, frustration, and occasional humor about the vaccination process.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731863260,
  "id": "chatcmpl-AUd2KfWZb4GIVRa0HuIHYqn6KnKJI",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_9b78b61c52",
  "usage": {
    "completion_tokens": 70,
    "completion_tokens_deta

## All 2016 negative

In [187]:
df_2016_neg.to_csv('../data/summary_2016_negative_tweets.csv')